In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import pandas as pd


In [32]:
from os import walk
from functools import reduce


def read_dataset():
    _, _, filenames = next(walk("labelled_dataset"))
    dfs = []
    for f in filenames:
        df = pd.read_csv(f"labelled_dataset/{f}")
        dfs.append(df)
        
    return pd.concat(dfs)

In [79]:
df = read_dataset()

In [35]:
df.head()

,Unnamed: 0,Unnamed: 0.1,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment_score,food_safety_flag
0,92461,92472,US,39315055,R2O55EFZHUL3L1,B00MDY0YVG,63674821,Lay's Cheddar Bacon Mac & Cheese Potato Chips ...,Grocery,1.0,1.0,3.0,N,N,If you're afraid you've been poisoned and don'...,These things - I refuse to call them &#34;chip...,2015-07-30,-0.4717,0
1,94800,94811,US,50817445,R30VZHOSGHZFTY,B007IW305A,118841434,Ajika Regal Pilaf with Nuts and Rasins -Whole ...,Grocery,1.0,1.0,1.0,N,Y,"Terrible, buggy, rotten smell",What a different experience the other person w...,2015-07-29,-0.6734,1
2,98590,98601,US,42738111,R3OMR3IWMCHDTH,B00II6YIGU,167483393,Capstar Dogs & Cats Blue 2-25lbs 6 pk Flea Tre...,Grocery,4.0,0.0,0.0,N,Y,This is a good product. My cats had no ill eff...,This is a good product. My cats had no ill ef...,2015-07-28,-0.2732,0
3,108424,108435,US,19278223,RGI0BTFNDROU9,B004OVWQDA,647776976,"Taste Nirvan, Coconut Water, (Pack of 12)",Grocery,3.0,5.0,5.0,N,Y,... return two of them because they did not ta...,This stuff is delicious and I have ordered sev...,2015-07-25,-0.7364,1
4,110787,110798,US,30804716,R3AS6OXW82P248,B00VN76PCQ,584830304,Crunchies Freeze Dried Fruit Snack Organic Var...,Grocery,3.0,0.0,1.0,N,Y,Disappointing and dangerous for toddlers,These were rather disappointing - they are ve...,2015-07-24,-0.8437,1


89


In [115]:
def build_dataset(df, review_feature, meta_features, label_feature):
    df.reset_index(inplace=True, drop=True)
    v = TfidfVectorizer()
    x = v.fit_transform(df[review_feature])
    tfidf_dataset = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
    meta_feature_dataset = df[meta_features]
                    # because sometimes the label are written as "0//" instead of just "0".This filter maps it correctly 
    return pd.concat([meta_feature_dataset, tfidf_dataset],axis=1), [0 if "0" in str(label) else 1 for label in df[label_feature]]



    

In [116]:
X, y = build_dataset(df, "review_body", ["sentiment_score", "star_rating"], "food_safety_flag" )

In [107]:
from sklearn.model_selection import train_test_split

In [117]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [119]:
from sklearn.naive_bayes import MultinomialNB

sgd = SGDClassifier(
    max_iter            = 1000, 
    tol                 = 1e-3,
    validation_fraction = 0.2
)

sgd.fit(X_train, y_train)

SGDClassifier(validation_fraction=0.2)

In [120]:
y_pred = sgd.predict(X_val)

In [121]:
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_val, y_pred)

print('\n Accuracy: ', accuracy_score(y_val, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.6666666666666666

Classification Report

               precision    recall  f1-score   support

           0       0.75      0.60      0.67        10
           1       0.60      0.75      0.67         8

    accuracy                           0.67        18
   macro avg       0.68      0.68      0.67        18
weighted avg       0.68      0.67      0.67        18

